In [1]:
import pathlib
import pandas as pd
import numpy as np
import pickle
import collections as co
import os
from datetime import datetime
from asammdf import MDF

ldf is not supported
xls is not supported
xlsx is not supported


In [57]:
union=np.array(['DTC1_CM','DTC1_VCU','DTC1_CCM','CF_Fcu_CbvSdCmd','CF_Hmu1_LoFuelWrn','CF_Hmu1_TMUDAQReq','CF_VCU_VehSt','BMM2_num_MinCellTemp','PCB_Temp_Slv'])
ser=np.array(['BMM2_num_MinCellTemp','PCB_Temp_Slv'])
cat=np.array(['DTC1_CM','DTC1_VCU','DTC1_CCM','CF_Fcu_CbvSdCmd','CF_Hmu1_LoFuelWrn','CF_Hmu1_TMUDAQReq','CF_VCU_VehSt'])

np.save('//192.168.0.67/hyundai/temp/union_list.npy',union)
np.save('//192.168.0.67/hyundai/temp/ser_list.npy',ser)
np.save('//192.168.0.67/hyundai/temp/cat_list.npy',cat)

In [7]:
def mdf2csv1(mdf_path,save_path):
    
    print(datetime.now())
    
    mdf_path=mdf_path.rstrip('/')
    var_list=np.load('//192.168.0.67/hyundai/temp/union_list.npy')
    ser=np.load('//192.168.0.67/hyundai/temp/ser_list.npy')
    cat=np.load('//192.168.0.67/hyundai/temp/cat_list.npy')
    mdf_list=list(pathlib.Path(mdf_path).glob('*.dat'))
    
    with open('//192.168.0.67/hyundai/pickle/sd_final.pkl','rb') as f:
        sd=pickle.load(f)
        
    print('start to make csv !')
    total=co.deque([])
    zero=[]
    
    for idx,var in enumerate(var_list,start=1):
        print('[{}/{}] {:>32s}'.format(idx,len(var_list),var),end='\r')
        time=np.array([])
        sample=np.array([])
        
        for mdf in mdf_list:
            data=MDF(mdf)
            start_time=data.start_time
            locs=data.whereis(var)
            for loc in locs:
                meta=data.get(group=loc[0],index=loc[1])
                time=np.append(time,np.array(start_time+np.round(meta.timestamps*1000,0)*pd.offsets.Milli()))
                sample=np.append(sample,meta.samples)
                
        if len(sample)==0:
            print(var,'is not collected in this car. It will be replaced to 0')
            zero.append(var)
            continue
            
        df=pd.DataFrame(sample,columns=[var],index=time)
        df.index.name='Timestamps'
        df.index=pd.to_datetime(df.index)
        df.sort_index(inplace=True)
        
        try:
            df[var]=df[var].map(float)
            
        except:
            try:
                df[var]=df[var].replace(sd[var])
                df=df.resample('1s').apply(lambda x:x.value_counts().index[0] if x.notnull().any() else np.NaN)
            except:
                print('String type feature "{}" replace failed. It will be replaced to 0'.format(var))
                zero.append(var)
                continue
                
        if var in cat:
            df=df.resample('1s').agg(lambda x:x.value_counts().index[0] if x.notnull().any() else np.NaN)
            
        elif var in ser:
            df=df.resample('1s').mean()
            
        df.index=df.index.strftime('%Y-%m-%d %H:%M:%S')
        total.append(df)
        
    merge_df=pd.concat(total,axis=1)
    merge_df.sort_index(inplace=True)
    merge_df.dropna(how='all',inplace=True)
    merge_df[zero]=np.array([0]*len(zero)+[np.NaN]*len(merge_df)*(len(zero)-1)).reshape(len(merge_df),-1)
    
    print('generate csv done !')
    
    #if not pathlib.Path(save_path).exists():
    #    os.mkdir(save_path)
        
    #merge_df.to_csv(save_path+'/merge_{}.csv'.format(mdf_path.split('/')[-1]))
    #print('save csv done !')
    print(datetime.now())
    
    return merge_df

In [8]:
merge_df1=mdf2csv1('//192.168.0.67/hyundai/MDF_Data/220201_51호차','//192.168.0.67/hyundai/CSV')

##1분 27초

2023-03-23 16:48:53.218112
start to make csv !
generate csv done !       PCB_Temp_Slv
2023-03-23 16:50:20.244287


In [35]:
def mdf2csv2(mdf_path,save_path):
    
    print(datetime.now())
    
    mdf_path=mdf_path.rstrip('/')
    var_list=np.load('//192.168.0.67/hyundai/temp/union_list.npy')
    ser=np.load('//192.168.0.67/hyundai/temp/ser_list.npy')
    cat=np.load('//192.168.0.67/hyundai/temp/cat_list.npy')
    mdf_list=list(pathlib.Path(mdf_path).glob('*.dat'))
    
    with open('//192.168.0.67/hyundai/pickle/sd_final.pkl','rb') as f:
        sd=pickle.load(f)
        
    print('start to make csv !')
    zero=[]
    total_dict={}
    
    for var in var_list:
        total_dict[var]=(co.deque([]),co.deque([]))
        #(timestamps,samples)
    
    for idx,mdf in enumerate(mdf_list,start=1):
        print('[{}/{}] file start'.format(idx,len(mdf_list)))
        data=MDF(mdf)
        start_time=data.start_time
        for var in var_list:
            locs=data.whereis(var)
            for loc in locs:
                meta=data.get(group=loc[0],index=loc[1])
                total_dict[var][0].extend(start_time+np.round(meta.timestamps*1000,0)*pd.offsets.Milli())
                total_dict[var][1].extend(meta.samples)
                
    total=co.deque([])
    for var,item in total_dict.items():
        df=pd.DataFrame({var:item[1]},index=item[0])
        df.index.name='Timestamps'
        df.sort_index(inplace=True)
        
        if len(df)==0:
            zero.append(var)
            print(var,'is not collected in this car. It will be replaced to 0')
            continue
            
        try:
            df[var]=df[var].map(float)
            
        except:
            try:
                df[var]=df[var].replace(sd[var])
                df=df.resample('1s').apply(lambda x:x.value_counts().index[0] if x.notnull().any() else np.NaN)
            except:
                print('String type feature "{}" replace failed. It will be replaced to 0'.format(var))
                zero.append(var)
                continue
            
        if var in cat:
            df=df.resample('1s').agg(lambda x:x.value_counts().index[0] if x.notnull().any() else np.NaN)
            
        elif var in ser:
            df=df.resample('1s').mean()
            
        df.index=df.index.strftime('%Y-%m-%d %H:%M:%S')
        total.append(df)
            
    merge_df=pd.concat(total,axis=1)
    merge_df.sort_index(inplace=True)
    merge_df.dropna(how='all',inplace=True)
    merge_df[zero]=np.array([0]*len(zero)+[np.NaN]*len(merge_df)*(len(zero)-1)).reshape(len(merge_df),-1)
    
    print('generate csv done !')
    
    #if not pathlib.Path(save_path).exists():
    #    os.mkdir(save_path)
        
    #merge_df.to_csv(save_path+'/merge_{}.csv'.format(mdf_path.split('/')[-1]))
    #print('save csv done !')
    print(datetime.now())       
        
    return merge_df

In [3]:
merge_df2=mdf2csv2('//192.168.0.67/hyundai/MDF_Data/220201_51호차','//192.168.0.67/hyundai/CSV')

##1분 27초

2023-03-23 16:07:43.711226
start to make csv !
generate csv done !       PCB_Temp_Slv
2023-03-23 16:09:10.688503


In [36]:
merge_df3=mdf2csv2('//192.168.0.67/hyundai/MDF_Data/220201_51호차','//192.168.0.67/hyundai/CSV')


2023-03-23 17:07:09.572287
start to make csv !
[1/7] file start
[2/7] file start
[3/7] file start
[4/7] file start
[5/7] file start
[6/7] file start
[7/7] file start
generate csv done !
2023-03-23 17:08:37.980303


In [29]:
merge_df1['CF_Fcu_CbvSdCmd'][merge_df1['CF_Fcu_CbvSdCmd']!=merge_df3['CF_Fcu_CbvSdCmd']]

Timestamps
2022-02-02 03:32:31    1.0
2022-02-02 03:32:32    1.0
2022-02-02 03:46:57    1.0
2022-02-02 07:00:04    2.0
2022-02-02 07:00:07    2.0
2022-02-02 07:04:24    1.0
Name: CF_Fcu_CbvSdCmd, dtype: float64

In [25]:
merge_df3['CF_Fcu_CbvSdCmd'][merge_df1['CF_Fcu_CbvSdCmd']!=merge_df3['CF_Fcu_CbvSdCmd']]

Timestamps
2022-02-02 03:32:31    0.0
2022-02-02 03:32:32    0.0
2022-02-02 03:46:57    0.0
2022-02-02 07:00:04    0.0
2022-02-02 07:00:07    0.0
2022-02-02 07:04:24    0.0
Name: CF_Fcu_CbvSdCmd, dtype: float64